In [ ]:
# Імпорт бібліотек
import pandas as pd
import matplotlib.pyplot as plt

# Зчитування агрегованого датасету
df = pd.read_csv('data.csv')

# Перевірка перших рядків
display(df.head())

# Візуалізація
plt.figure(figsize=(10, 6))
plt.plot(df['date'], df['unit_sales'])
plt.title('Unit Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Unit Sales')
plt.xticks(df['date'][::200], rotation=45)
plt.grid()
plt.tight_layout()
plt.show()